In [51]:
from src.utils import utils
from src.utils.utils import categories
import json
import random

In [57]:
def prepare_fewshot(dataset, seed):
    now = gold_data = utils.load_hf_dataset(path=f"/data/mourad/narratives/sft_data", dataset=dataset)['train'].shuffle(seed=seed)
    pq = gold_data = utils.load_hf_dataset(path=f"/data/mourad/narratives/sft_data", dataset=dataset)['train'].shuffle(seed=seed)
    cats = categories['cause_category'] + categories['effect_category']

    fewshot = []
    cats_included = []
    nons_included = 0
    for instance in now:
        gold = json.loads(utils.reconstruct_training_input(instance))
        gold = json.dumps(gold)
        remaining_cats = set(cats).difference(cats_included)
        for cat in remaining_cats:
            if cat in gold:
                cats_included.append(cat)
                fewshot.append(f"{instance['text']}\n{gold}")
                break
        # print(gold)
        if nons_included < 5 and 'null' in gold: 
            nons_included += 1
            fewshot.append(f"{instance['text']}\n{gold}")       

    random.seed(seed)
    random.shuffle(fewshot)
    with open(f'../prompt-templates/in-context/gpt/fewshot_{dataset}_{seed}.txt', 'w') as f:
        # shuffle now_fewshot
        for instance in fewshot:
            f.write(instance + '\n\n')

In [58]:
for seed in [1,2,3,4]:
    prepare_fewshot('now', seed)
    prepare_fewshot('proquest', seed)
    prepare_fewshot('now_and_proquest', seed)